In [ ]:
#| default_exp gym

In [ ]:
#| export
import jax
import jax.numpy as jnp
from gymnax.environments import environment, spaces
from typing import Tuple, Optional
from functools import partial
from jax.tree_util import tree_map

# Assuming the 'chewc' library files are in the path
from chewc.structs import Population, BreedingState, Trait, GeneticMap, quick_haplo, add_trait
from chewc.pheno import calculate_phenotypes
from chewc.cross import random_mating, cross_pair

class StoaEnv(environment.Environment):
    """
    A Gymnax environment for the ChewC breeding simulation.
    
    The agent's goal is to maximize the genetic gain of the population
    over a fixed number of generations by choosing the selection intensity at each step.
    """

    def __init__(self, n_founders=100, n_pop_size=200, n_chr=5, n_loci=1000, n_qtl_per_chr=50, total_gen=20, max_crossovers=10):
        super().__init__()
        self.n_founders = n_founders
        self.n_pop_size = n_pop_size
        self.total_gen = total_gen
        self.max_crossovers = max_crossovers
        self.n_chr = n_chr
        self.n_loci = n_loci
        self.n_qtl_per_chr = n_qtl_per_chr
        
        key = jax.random.PRNGKey(42)
        pop_key, trait_key = jax.random.split(key)

        # Founder pop is created once and reused for every reset
        self.founder_pop, self.genetic_map = quick_haplo(
            key=pop_key, n_ind=self.n_founders, n_chr=self.n_chr, seg_sites=self.n_loci
        )

        self.trait_architecture = add_trait(
            key=trait_key,
            founder_pop=self.founder_pop,
            n_qtl_per_chr=self.n_qtl_per_chr,
            mean=jnp.array([100.0]),
            var_a=jnp.array([10.0]),
            var_d=jnp.array([0.0]),
            sigma=jnp.array([[1.0]])
        )
        self.heritabilities = jnp.array([0.5])

    @property
    def default_params(self):
        return {}

    def step_env(self, key: jax.Array, state: BreedingState, action: jax.Array, params) -> Tuple[jax.Array, BreedingState, float, bool, dict]:
        key, pheno_key, mating_key, cross_key = jax.random.split(key, 4)
        current_pop = state.population
        
        selection_proportion = (action[0] + 1) / 2 * (0.95 - 0.05) + 0.05
        
        phenotypes, _ = calculate_phenotypes(
            key=pheno_key,
            population=current_pop,
            trait=self.trait_architecture,
            heritability=self.heritabilities
        )
        
        n_current_pop = current_pop.geno.shape[0]
        n_select = (n_current_pop * selection_proportion).astype(jnp.int32)
        n_select = jnp.maximum(2, n_select)
        
        sorted_indices = jnp.argsort(-phenotypes[:, 0].flatten())
        sorted_parent_pool = tree_map(lambda x: x[sorted_indices], current_pop)

        pairings = random_mating(mating_key, n_parents=n_select, n_crosses=self.n_pop_size)
        mother_indices, father_indices = pairings[:, 0], pairings[:, 1]
        
        mothers_geno = sorted_parent_pool.geno[mother_indices]
        fathers_geno = sorted_parent_pool.geno[father_indices]
        mothers_ibd = sorted_parent_pool.ibd[mother_indices]
        fathers_ibd = sorted_parent_pool.ibd[father_indices]
        
        vmapped_cross = jax.vmap(cross_pair, in_axes=(0, 0, 0, 0, 0, None, None))
        offspring_keys = jax.random.split(cross_key, self.n_pop_size)
        offspring_geno, offspring_ibd = vmapped_cross(
            offspring_keys, mothers_geno, fathers_geno, mothers_ibd, fathers_ibd, self.genetic_map, self.max_crossovers
        )
        
        new_generation = state.generation + 1
        new_ids = jnp.arange(self.n_pop_size, dtype=jnp.int32) + state.next_id
        new_meta = jnp.stack([
            new_ids,
            sorted_parent_pool.meta[mother_indices, 0],
            sorted_parent_pool.meta[father_indices, 0],
            jnp.full((self.n_pop_size,), new_generation, dtype=jnp.int32),
        ], axis=-1)
        
        new_population = Population(geno=offspring_geno, ibd=offspring_ibd, meta=new_meta)
        
        next_state = BreedingState(
            population=new_population, key=key, generation=new_generation, next_id=state.next_id + self.n_pop_size
        )
        
        done = next_state.generation >= self.total_gen

        def get_final_reward(final_state: BreedingState):
            reward_key, _ = jax.random.split(final_state.key)
            _, tbvs = calculate_phenotypes(
                reward_key, final_state.population, self.trait_architecture, self.heritabilities
            )
            return jnp.mean(tbvs[:, 0])

        reward = jax.lax.cond(done, get_final_reward, lambda s: 0.0, operand=next_state)
        
        obs = self._get_obs(next_state)
        return obs, next_state, reward, done, {}

    def reset_env(self, key: jax.Array, params) -> Tuple[jax.Array, BreedingState]:
        """
        Resets the environment. Includes a "burn-in" step to ensure the
        initial population size matches the ongoing population size, which is
        required for JAX's conditional logic in auto-resetting environments.
        """
        # Create a new state with the small founder population at generation 0
        burn_in_key, pheno_key, mating_key, cross_key = jax.random.split(key, 4)
        founder_state = BreedingState(
            population=self.founder_pop, key=burn_in_key, generation=0, next_id=self.n_founders
        )
        
        # --- Perform a single burn-in breeding step to reach `n_pop_size` ---
        phenotypes, _ = calculate_phenotypes(
            pheno_key, founder_state.population, self.trait_architecture, self.heritabilities
        )
        
        # Select top 50% of founders by default for the first cross
        n_select = jnp.maximum(2, self.n_founders // 2)
        sorted_indices = jnp.argsort(-phenotypes[:, 0].flatten())
        sorted_founders = tree_map(lambda x: x[sorted_indices], founder_state.population)
        
        pairings = random_mating(mating_key, n_parents=n_select, n_crosses=self.n_pop_size)
        mother_indices, father_indices = pairings[:, 0], pairings[:, 1]
        
        mothers_geno = sorted_founders.geno[mother_indices]
        fathers_geno = sorted_founders.geno[father_indices]
        mothers_ibd = sorted_founders.ibd[mother_indices]
        fathers_ibd = sorted_founders.ibd[father_indices]

        vmapped_cross = jax.vmap(cross_pair, in_axes=(0, 0, 0, 0, 0, None, None))
        offspring_keys = jax.random.split(cross_key, self.n_pop_size)
        offspring_geno, offspring_ibd = vmapped_cross(
            offspring_keys, mothers_geno, fathers_geno, mothers_ibd, fathers_ibd, self.genetic_map, self.max_crossovers
        )
        
        # This is the actual initial state for the agent at generation 1
        initial_gen = 1
        initial_ids = jnp.arange(self.n_pop_size, dtype=jnp.int32) + founder_state.next_id
        initial_meta = jnp.stack([
            initial_ids,
            sorted_founders.meta[mother_indices, 0],
            sorted_founders.meta[father_indices, 0],
            jnp.full((self.n_pop_size,), initial_gen, dtype=jnp.int32),
        ], axis=-1)
        
        initial_pop = Population(geno=offspring_geno, ibd=offspring_ibd, meta=initial_meta)
        
        # The state returned to the agent
        initial_state = BreedingState(
            population=initial_pop, key=key, generation=initial_gen, next_id=founder_state.next_id + self.n_pop_size
        )

        obs = self._get_obs(initial_state)
        return obs, initial_state

    def _get_obs(self, state: BreedingState) -> jax.Array:
        remaining_gen = jnp.maximum(0.0, (self.total_gen - state.generation) / self.total_gen)
        pheno_key, _ = jax.random.split(state.key)
        phenotypes, _ = calculate_phenotypes(
            key=pheno_key, population=state.population, trait=self.trait_architecture, heritability=self.heritabilities
        )
        pheno_trait_1 = phenotypes[:, 0]
        genetic_mean = jnp.mean(pheno_trait_1)
        genetic_var = jnp.var(pheno_trait_1)
        return jnp.array([remaining_gen, genetic_mean, genetic_var])

    def observation_space(self, params):
        low = jnp.array([0.0, -jnp.inf, 0.0], dtype=jnp.float32)
        high = jnp.array([1.0, jnp.inf, jnp.inf], dtype=jnp.float32)
        return spaces.Box(low, high, (3,), dtype=jnp.float32)

    def action_space(self, params):
        return spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=jnp.float32)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()